In [72]:
import datasets
import utils
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
import pandas as pd
import pickle_utils as pu
import imp
import GPy

In [89]:
#dset = datasets.datasets()["BostonHousing"][0].astype(np.float64)
dset = pu.load("impute_benchmark/amputed_BostonHousing_MCAR_rows_0.3.pkl.gz")
mog = pu.load("impute_benchmark/imputed_BGMM_20_BostonHousing_MCAR_rows_0.3/params.pkl.gz")
mog['means'] = mog['means'][:, 1:]
mog['covariances'] = mog['covariances'][:, 1:, 1:]

test_mask = np.random.rand(len(dset)) < 0.2

norm_mean = dset[~test_mask].mean()
norm_std = dset[~test_mask].std()

dset = (dset - norm_mean) / norm_std
dset_nan = dset.copy()
dset = dset.fillna(0)
dset_var = dset.applymap(lambda x: 1 if np.isnan(x) else 0)

y_train = dset.values[~test_mask, 0]
X_train = dset.values[~test_mask, 1:]
X_train_var = dset_var.values[~test_mask, 1:]
X_train_nan = dset_nan.values[~test_mask, 1:]
y_test = dset.values[test_mask, 0]
X_test = dset.values[test_mask, 1:]
X_test_var = dset_var.values[test_mask, 1:]
X_test_nan = dset_nan.values[test_mask, 1:]

In [3]:
def r_squared(y_true, y):
    u = np.sum((y-y_true)**2)
    v = np.sum((y_true - np.mean(y_true))**2)
    return 1 - u/v
def rmse(y_true, y):
    return np.mean((y_true - y)**2)

In [4]:
np.all(np.isfinite(y_train)), np.all(np.isfinite(X_train)), np.all(np.isfinite(X_train_nan)), np.all(np.isfinite(y_test)), np.all(np.isfinite(X_test)), np.all(np.isfinite(X_test_nan))

(True, True, False, True, True, False)

In [90]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1,
                           max_features=int(np.floor(X_train.shape[1])/3),
                           bootstrap=False,
                           min_samples_split=5)
rf.fit(X_train, y_train)
train_perf = rf.score(X_train, y_train)
test_perf = rf.score(X_test, y_test)
print("Test: {:.4f}, Training: {:.4f}".format(test_perf, train_perf))
y = rf.predict(X_train)
print("Train:", rmse(y_train, y.flatten()))
y = rf.predict(X_test)
print("Test:", rmse(y_test, y.flatten()))

Test: 0.2677, Training: 0.9549
Train: 0.0315987105856
Test: 2.34436448443


In [286]:
import GPy
m = GPy.models.GPRegression(X_train, y_train[:,np.newaxis],
                            kernel=GPy.kern.Matern32(X_train.shape[1], ARD=True) + GPy.kern.White(X_train.shape[1]))
#m.optimize()
y, _ = m.predict(X_train)
print("Train:", rmse(y_train, y.flatten()))
y, _ = m.predict(X_test)
print("Test:", rmse(y_test, y.flatten()))

Train: 0.262916908349
Test: 1.22404203677


In [214]:
m.kern

sum.,value,constraints,priors
rbf.variance,0.55463044994,+ve,
rbf.lengthscale,2.04190842862,+ve,
white.variance,0.165010421819,+ve,


In [104]:
imp.reload(gmm_impute)
gmm_impute._gmm_impute(mog, X_train_nan[9:10], 1)

(array([[[-0.42006788,  1.01905536,  0.68299323, -0.40612554,  0.63205938,
          -0.5832783 ,  0.39105928,  0.67011543,  0.35313276, -0.21148562,
           0.3739001 ,  1.10710231]]]),
 array([[ 0.63855337,  0.32716695,  0.        ,  1.04207718, -0.0528659 ,
          1.04048448,  0.61910684,  0.2997959 ,  0.54710433,  1.24750701,
          0.30055021,  0.        ]]))

In [105]:
X_train_nan[9:10]

array([[        nan,         nan,  0.68299323,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,  1.10710231]])

In [109]:
imp.reload(mog_rbf)
d = mog_rbf.uncertain_point(X_train_nan[9], mog, 1., np.eye(12), single_gaussian_moment_matching=True)
d['means'], d['covariances']

(array([[[-0.42006788],
         [ 1.01905536],
         [-0.40612554],
         [ 0.63205938],
         [-0.5832783 ],
         [ 0.39105928],
         [ 0.67011543],
         [ 0.35313276],
         [-0.21148562],
         [ 0.3739001 ]]]),
 array([[[ 0.63855337,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.32716695,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  1.04207718,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , -0.0528659 ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,  1.04048448,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
  

In [66]:
imp.reload(mog_rbf)
imp.reload(gmm_impute)
uk2 = mog_rbf.UncertainMoGRBFWhite(X_train.shape[1], mog, cutoff=1.0, single_gaussian=True)
K_uk2 = uk2.K(X_train_nan, X_train_nan)

To compute map took me: 0.4738774299621582
FOR REAL [[[-0.15875905]
  [ 0.17830032]
  [-0.41094708]
  [-0.40360746]]]


In [110]:
imp.reload(mog_rbf)
imp.reload(gmm_impute)
uk3 = mog_rbf.UncertainGaussianRBFWhite(X_train.shape[1], mog)
K_uk3 = uk3.K(X_train_nan, X_train_nan)

To compute map (rectangular) took me: 0.9276993274688721
To compute matrix took me: 0.2105269432067871
(408, 408)


In [69]:
np.max(K_uk3 - K_uk3.T)

0.00090383853741499674

In [111]:
import mog_rbf
import imp
import gmm_impute
imp.reload(gmm_impute)
imp.reload(mog_rbf)

uncertain_kern = mog_rbf.UncertainMoGRBFWhite(X_train.shape[1], mog)
uncertain_k = GPy.models.GPRegression(X_train_nan, y_train[:, np.newaxis],
                                     kernel=uncertain_kern)
y, _ = uncertain_k.predict(X_train_nan)
print("Train:", rmse(y_train, y.flatten()))
y, _ = uncertain_k.predict(X_test_nan)
print("Test:", rmse(y_test, y.flatten()))

To compute map took me: 0.38788938522338867
To compute matrix took me: 5.707479238510132
To compute map took me: 0.38481831550598145
To compute matrix took me: 11.051551580429077
Train: 0.303080763536
To compute map took me: 0.4139392375946045
To compute matrix took me: 2.8104450702667236
Test: 2.51814645194


In [112]:
import mog_rbf
import imp
import gmm_impute
imp.reload(gmm_impute)
imp.reload(mog_rbf)

uncertain_kern = mog_rbf.UncertainGaussianRBFWhite(X_train.shape[1], mog)
uncertain_k = GPy.models.GPRegression(X_train_nan, y_train[:, np.newaxis],
                                     kernel=uncertain_kern)
y, _ = uncertain_k.predict(X_train_nan)
print("Train:", rmse(y_train, y.flatten()))
y, _ = uncertain_k.predict(X_test_nan)
print("Test:", rmse(y_test, y.flatten()))

To compute map took me: 0.47240662574768066
To compute matrix took me: 0.21442580223083496
(408, 408)
To compute map (rectangular) took me: 0.9931590557098389
To compute matrix took me: 0.2064647674560547
(408, 408)
Train: 0.323577201919
To compute map (rectangular) took me: 0.6183946132659912
To compute matrix took me: 0.0666952133178711
(408, 98)
Test: 2.51151833254


In [270]:
X_train_alt = X_train_nan.copy()
X_test_alt = X_test_nan.copy()

for i, X in enumerate(X_train_alt):
    mask = np.isnan(X)
    d = gmm_impute.conditional_mog(mog, X, mask)
    # Perform moment matching to get mean and variance
    # https://stats.stackexchange.com/questions/16608/what-is-the-variance-of-the-weighted-mixture-of-two-gaussians
    ms = d['means'] * d['weights'][:, np.newaxis]
    mean = np.sum(ms, axis=0)
    var_mix = np.sum(d['covariances'] * np.eye(d['covariances'].shape[1]), axis=2).sum(axis=0)
    var = var_mix + np.sum(ms * d['means'], axis=0) - mean
    X_train_alt[i, mask] = mean
    X_train_var[i, mask] = var

for i, X in enumerate(X_test_alt):
    mask = np.isnan(X)
    d = gmm_impute.conditional_mog(mog, X, mask)
    X_test_alt[i, mask] = np.sum(d['means'] * d['weights'][:, np.newaxis], axis=0)


sparse_m = GPy.models.SparseGPRegression(
    X_train_alt, y_train[:, np.newaxis],
    kernel=(GPy.kern.RBF(X_train.shape[1], ARD=False)
                   + GPy.kern.White(X_train.shape[1])),
    Z=X_train_alt,
    X_variance=X_train_var)
#sparse_m.optimize()
y, _ = sparse_m.predict(X_train_alt)
print("Train:", rmse(y_train, y.flatten()))
y, full_cov = sparse_m.predict(X_test, full_cov=True)
print("Test:", rmse(y_test, y.flatten()))

 /home/adria/venv/lib/python3.5/site-packages/paramz/transformations.py:109: RuntimeWarning:divide by zero encountered in log


Train: 0.361617084375
Test: 0.0740350690732


In [271]:
full_cov

array([[  2.78635832e+00,   2.06344958e-04,   1.64838064e-05, ...,
          4.35285552e-04,   1.00000000e-15,   8.26572097e-04],
       [  2.06344958e-04,   2.94234875e+00,   1.27033141e-03, ...,
          4.72627943e-01,   1.00000000e-15,   1.87282191e-02],
       [  1.64838064e-05,   1.27033141e-03,   2.78212823e+00, ...,
          8.27680241e-04,   2.87765037e-03,   1.40391869e-02],
       ..., 
       [  4.35285552e-04,   4.72627943e-01,   8.27680241e-04, ...,
          2.93926028e+00,   1.00000000e-15,   5.84986024e-02],
       [  1.00000000e-15,   1.00000000e-15,   2.87765037e-03, ...,
          1.00000000e-15,   2.71902756e+00,   1.00000000e-15],
       [  8.26572097e-04,   1.87282191e-02,   1.40391869e-02, ...,
          5.84986024e-02,   1.00000000e-15,   2.80199682e+00]])

In [39]:
import selu
import denoising_ae as dae
import imp
imp.reload(dae)
imp.reload(selu)

tf_float = tf.float32
tf.reset_default_graph()

nn_X = tf.placeholder(tf_float, shape=[None, X_train.shape[1]], name="X")
nn_y = tf.placeholder(tf_float, shape=[None, 1], name="y")
nn_kp = tf.placeholder(tf_float, shape=[], name="keep_prob")
nn_lr = tf.placeholder(tf_float, shape=[], name="learning_rate")


nn_preds = dae.FC_net(nn_X, [(1024, selu.nlin)]*8 + [(1, None)], selu.initializer, keep_prob=nn_kp)
nn_loss = tf.reduce_mean(tf.squared_difference(nn_preds, nn_y))
nn_train = tf.train.GradientDescentOptimizer(nn_lr).minimize(nn_loss)

In [40]:
from tqdm import tqdm
import itertools

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

batch_size = 128

num_batches = int(np.ceil(X_train.shape[0] / batch_size))

pbar = tqdm(itertools.count(0))
for step in pbar:
    i = step % num_batches
    # Populate feed_dict; duplicated code for num and cat
    s = slice(batch_size * i, batch_size * (i + 1))
    feed_dict = {nn_kp: 0.95,
                 nn_lr: 0.001 if step < 800 else 0.0001,
                 nn_X: X_train[s],
                 nn_y: y_train[s, np.newaxis],
                }
    del i
    
    if step % 100 == 0:
        _, l, y = sess.run([nn_train, nn_loss, nn_preds], feed_dict)
        d = "Loss: {} ".format(l)
        d += "Train: {} ".format(r_squared(y_train[s], y.ravel()))
        y = sess.run(nn_preds, {nn_kp: 1.0,
                                nn_X: X_test,
                                nn_y: y_test[:, np.newaxis]})
        d += "Test: {} ".format(r_squared(y_test, y.ravel()))
        pbar.set_description(d)
    else:
        sess.run(nn_train, feed_dict)


0it [00:00, ?it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 1it [00:00,  3.25it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 3it [00:00,  4.23it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 5it [00:00,  5.52it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 8it [00:00,  7.19it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 11it [00:00,  9.28it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 14it [00:00, 11.47it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 17it [00:01, 13.87it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 20it [00:01, 16.15it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 23it [00:01, 18.33it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 

KeyboardInterrupt: 

In [41]:
tf.summary.FileWriter("faceoff/", graph=sess.graph)

Loss: 0.055597126483917236 Train: 0.9541863795146673 Test: 0.9235167656130201 : 3687it [02:48, 21.89it/s]

In [23]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
import time

# Fit KernelRidge with parameter selection based on 5-fold cross validation
param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [RBF(l)
                         for l in np.logspace(-2, 2, 10)]}
kr = GridSearchCV(KernelRidge(), cv=5, param_grid=param_grid)
stime = time.time()
kr.fit(X_train[:100], y_train[:100])
print("Time for KRR fitting: %.3f" % (time.time() - stime))

Time for KRR fitting: 0.549


In [18]:
Z = np.random.rand(1000, X_train.shape[1])
rbf = GPflow.kernels.RBF(X_train.shape[1], variance=1, ARD=True)
m = GPflow.sgpr.SGPR(X_train, y_train[:,None], kern=rbf, Z=Z)
def logger(x):
    if (logger.i % 10) == 0:
        print(x, m._objective(x)[0])
    logger.i+=1
logger.i = 0

m.optimize(method=tf.train.AdamOptimizer(), callback=logger)



[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 181586.72233875]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 180537.74127184]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 179500.65675795]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 178476.56694212]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 177466.08404523]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 176469.42592986]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 175486.5617425]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 174517.32678109]
Caught KeyboardInterrupt, setting model                  with most recent state.


In [24]:
y, _ = m.predict_y(X_train)
print("Train:", r_squared(y_train, y.flatten()))
y, _ = m.predict_y(X_test)
print("Test:", r_squared(y_test, y.flatten()))

Train: -4.42959616572
Test: -4.39652614283
